# Laboratorio 2

- Jaime Andres Torres Bermejo - 202014866
- Elkin Rafael Cuello - 202215037

## Problema 1: Redes de Transporte

La temporada navideña es uno de los periodos de mayor actividad
comercial en Colombia. Durante este tiempo, la demanda de productos como juguetes,
ropa, electrodomésticos, y alimentos tradicionales aumenta considerablemente en
diversas regiones del país. Dos de las principales ciudades distribuidoras, Bogotá y
Medellín, se preparan para abastecer a otras ciudades importantes como Cali,
Barranquilla, Pasto, Tunja, Chia y Manizales.
Cada ciudad de destino tiene una demanda específica de productos para satisfacer las
necesidades de sus habitantes durante la temporada. Mientras, Bogotá y Medellín
tienen una oferta limitada de productos por las restricciones en la cadena de suministro.
La eficiencia en la distribución es crucial para garantizar que todos los productos lleguen
a tiempo y se minimicen los costos de transporte.

### Entregables

- Plantee el modelo matematico del problema, Defina:
    - Conjuntos
    - Parámetros
    - Variable de Decisión
    - Función Objetivo
    - Restricciones
- Realize una implementación en PYOMO o en el software de su gusto
- Realize un análisis de Sensibilidad de las cuidades de destino y las cuidades de
origen.
- Mueva 50 toneladas de oferta de medellin a bogota y repita el analisis de
sensibilidad. ¿Que ha cambiado? Recomendaría este cambio or que otro cambio
puede proponer?

In [23]:
from pyomo.environ import *
import math
import numpy as np


# Modelo
Model = ConcreteModel()

# Conjuntos
origenes = ['Bogota', 'Medellin']
destinos = ['Cali', 'Barranquilla', 'Pasto', 'Tunja', 'Chia', 'Manizales']
costos = {
    ('Bogota', 'Cali'): 100, ('Medellin', 'Cali'): 2.5,
    ('Bogota', 'Barranquilla'): 2.5, ('Medellin', 'Barranquilla'): 100,
    ('Bogota', 'Pasto'): 1.6, ('Medellin', 'Pasto'): 2.0,
    ('Bogota', 'Tunja'): 1.4, ('Medellin', 'Tunja'): 1.0,
    ('Bogota', 'Chia'): 0.8, ('Medellin', 'Chia'): 1.0,
    ('Bogota', 'Manizales'): 1.4, ('Medellin', 'Manizales'): 0.8
}
demandas = {'Cali': 125, 'Barranquilla': 175, 'Pasto': 225, 'Tunja': 250, 'Chia': 225, 'Manizales': 200}
ofertas = {'Bogota': 550, 'Medellin': 700}

# Variables
Model.x = Var(origenes, destinos, within=NonNegativeReals)

# Función objetivo: minimizar los costos de transporte
def objetivo(model):
    return sum(model.x[i, j] * costos[i, j] for i in origenes for j in destinos)
Model.objetivo = Objective(rule=objetivo, sense=minimize)

# Restricción de oferta: no exceder la cantidad disponible en cada origen
def oferta_rule(model, i):
    return sum(model.x[i, j] for j in destinos) <= ofertas[i]
Model.oferta = Constraint(origenes, rule=oferta_rule)

# Restricción de demanda: satisfacer la demanda en cada destino
def demanda_rule(model, j):
    return sum(model.x[i, j] for i in origenes) == demandas[j]
Model.demanda = Constraint(destinos, rule=demanda_rule)

# Resolver el modelo
solver = SolverFactory('glpk')
solver.solve(Model)

Model.display()

Model unknown

  Variables:
    x : Size=12, Index={Bogota, Medellin}*{Cali, Barranquilla, Pasto, Tunja, Chia, Manizales}
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
          ('Bogota', 'Barranquilla') :     0 : 175.0 :  None : False : False : NonNegativeReals
                  ('Bogota', 'Cali') :     0 :   0.0 :  None : False : False : NonNegativeReals
                  ('Bogota', 'Chia') :     0 : 150.0 :  None : False : False : NonNegativeReals
             ('Bogota', 'Manizales') :     0 :   0.0 :  None : False : False : NonNegativeReals
                 ('Bogota', 'Pasto') :     0 : 225.0 :  None : False : False : NonNegativeReals
                 ('Bogota', 'Tunja') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('Medellin', 'Barranquilla') :     0 :   0.0 :  None : False : False : NonNegativeReals
                ('Medellin', 'Cali') :     0 : 125.0 :  None : False : False : NonNegativeReals
                ('Medell

### Analisis de sensibilidad - Problema 1

b. Realize un analísis de Sensibilidad de las cuidades de destino y las cuidades de origen

In [24]:
Model.dual = Suffix(direction=Suffix.IMPORT)
resultado = solver.solve(Model)

In [25]:
print("\nSolution")

for i in destinos:
    print(f"Dual demanda {i}: {Model.dual[Model.demanda[i]]}")

for j in origenes:
    print(f"Dual oferta {j}: {Model.dual[Model.oferta[j]]}")


Solution
Dual demanda Cali: 2.5
Dual demanda Barranquilla: 2.7
Dual demanda Pasto: 1.8
Dual demanda Tunja: 1.0
Dual demanda Chia: 1.0
Dual demanda Manizales: 0.8
Dual oferta Bogota: -0.2
Dual oferta Medellin: 0.0


c. Mueva 50 toneladas de oferta de medellin a bogota y repita el analisis de sensibilidad. ¿Que ha cambiado? Recomendaria este cambio or que otro cambio puede proponer?

Se modifican los parametros

In [26]:
ofertas['Bogota'] = ofertas['Bogota'] + 50
ofertas['Medellin'] = ofertas['Medellin'] - 50 

In [27]:
resultado = solver.solve(Model)

In [28]:
print("\nSolution")

for i in destinos:
    print(f"Dual demanda {i}: {Model.dual[Model.demanda[i]]}")

for j in origenes:
    print(f"Dual oferta {j}: {Model.dual[Model.oferta[j]]}")


Solution
Dual demanda Cali: 2.5
Dual demanda Barranquilla: 2.7
Dual demanda Pasto: 1.8
Dual demanda Tunja: 1.0
Dual demanda Chia: 1.0
Dual demanda Manizales: 0.8
Dual oferta Bogota: -0.2
Dual oferta Medellin: 0.0


¿Que ha cambiado? Recomendaria este cambio or que otro cambio puede proponer?

RTA: Al realizar este cambio, no se ha modificado ningun valor de el analisis de sensibilidad. La ausencia de cambios en el valor objetivo puede indicar que el problema no es sensible a los parámetros modificados. Esto puede deberse a que los parámetros no influyen directamente en la función objetivo o que sus efectos se cancelan.

# Problema 2: Rutas Óptimas para equipos de Inspección de Infrastructura en Colombia

## Variables

- i: city from which the team departs
- j: city to which the team arrives
- k: infra team 

## Restraints

- $
\text{Minimize:} \quad \sum_{i \in N} \sum_{j \in N} c_{ij} \cdot x_{ijk} \forall K
$

- $ x_{ijk} $ be a binary decision variable where:
  $$
  x_{ij} = \begin{cases} 
  1 & \text{if the traveler travels directly from city } i \text{ to city } j, \text{while being part of team 'k'} \\
  0 & \text{otherwise}.
  \end{cases}
  $$


In [57]:
from pyomo.environ import *
import math
import numpy as np
import pandas as pd

Model = ConcreteModel()

numNodes=6

N=RangeSet(1, numNodes)

c={(1,1):999, (1,2):1,   (1,3):1,   (1,4):2, (1,5):2,(1,6):1,\
      (2,1):1, (2,2):999,   (2,3):3,   (2,4):2, (2,5):3,(2,6):3,\
      (3,1):1, (3,2):3,   (3,3):999,   (3,4):3, (3,5):2,(3,6):3,\
      (4,1):2, (4,2):2,   (4,3):3,   (4,4):999, (4,5):1,(4,6):2,\
      (5,1):2, (5,2):3,   (5,3):3,   (5,4):1, (5,5):999,(5,6):3,\
      (6,1):1, (6,2):2,   (6,3):1,   (6,4):3, (6,5):3,(6,6):999} #costos
k = [1,2,3] #equipos
localidad = {
    1: [6,1],
    2: [2,1],
    3: [3,4,5,1]
}#conexiones
final = {1: 1, 2: 1, 3: 1}  # Localidad final
# VARIABLES****************************************************************************
Model.N = Set(initialize=N) 

Model.c = Param(Model.N, Model.N, initialize=c)  # Costos
Model.x = Var(Model.N,Model.N,k, domain=Binary) #v. decision
Model.k = Set(initialize=k)  # Equipos
Model.u = Var(Model.N,Model.k, domain=NonNegativeReals) #
Model.localidad = Set(Model.k, initialize=lambda Model, k: localidad[k])
Model.final = Param(Model.k, initialize=final, within=Model.N)  # Localidad de inicio/fin de cada equipo

def objective(Model):
    return sum(Model.c[i,j] * Model.x[i,j,k] for k in Model.k for i in Model.localidad[k] for j in Model.localidad[k])
Model.obj = Objective(rule=objective, sense=minimize)

def source_rule(Model,i,k):
    if i in Model.localidad[k]:
        return sum(Model.x[i, j, k] for j in Model.localidad[k] if j != i) == 1
    else:
        return Constraint.Skip
Model.source=Constraint(N,k, rule=source_rule)

def destination_rule(Model,j,k):
    if j==1:
        return sum(Model.x[i,j,k] for i in Model.localidad[k] if i != j) == 1
    else:
        return Constraint.Skip
Model.destination=Constraint(N,k, rule=destination_rule)

def MTZ_rule(Model, i, j, k):
    if i != j and i != Model.localidad[k] and j != Model.localidad[k] and i in Model.localidad[k] and j in Model.localidad[k]:
        return Model.u[i, k] - Model.u[j, k] + len(Model.localidad[k]) * Model.x[i, j, k] <= len(Model.localidad[k]) - 1
    else:
        return Constraint.Skip
Model.mtz=Constraint(N,N,k, rule=MTZ_rule)

def transit_rule(Model, i, k):
    if i != Model.final[k] and i in Model.localidad[k]:
        return sum(Model.x[i, j, k] for j in Model.localidad[k]) == sum(Model.x[j, i, k] for j in Model.localidad[k])
    else:
        return Constraint.Skip
Model.transit_rule = Constraint(Model.N, Model.k, rule=transit_rule)

# SOLVER******************************************************************
SolverFactory('glpk').solve(Model)

Model.display()


ERROR: Rule failed for Param 'final' with index 1: ValueError: Invalid
parameter value: final[1] = '0', value type=<class 'int'>.
            Value not in parameter domain N
ERROR: Constructing component 'final' from data=None failed:
        ValueError: Invalid parameter value: final[1] = '0', value type=<class
        'int'>.
    	Value not in parameter domain N


ValueError: Invalid parameter value: final[1] = '0', value type=<class 'int'>.
	Value not in parameter domain N

# Problema 3: Optimización de Sensores en Ciudades Inteligentes

## Planteamiento matemático del problema 3:

### Conjuntos
- **S**: Conjunto de sensores disponibles.
- **L**: Conjunto de localidades que necesitan ser monitoreadas.

### Parámetros
- **EC<sub>s</sub>**: Costo de consumo de energía del sensor *s*.
- **CC<sub>s,l</sub>**: Costo de comunicación entre el sensor *s* y la localidad *l*.
- **IC<sub>l</sub>**: Costo de instalación en la localidad *l*.
- **SC<sub>s,l</sub>**: Cobertura del sensor *s* en la localidad *l* (1 si el sensor puede cubrir la localidad, 0 en caso contrario).
- **ZC<sub>l,l'</sub>**: Cobertura de zona entre las localidades *l* y *l'* (1 si las localidades son adyacentes, 0 en caso contrario).

### Variables de Decisión
- **x<sub>s,l</sub>**: Variable binaria que indica si el sensor *s* está instalado en la localidad *l* (1 si está instalado, 0 en caso contrario).
- **y<sub>s</sub>**: Variable binaria que indica si el sensor *s* está instalado en cualquier localidad (1 si está instalado, 0 en caso contrario).

### Función Objetivo
Minimizar el costo total:

$$
\min \space ( \sum_{s \in S} EC_{s} \cdot y_{s} + \sum_{s \in S} \sum_{l \in L} CC_{s,l} \cdot x_{s,l} + \sum_{s \in S} \sum_{l \in L} IC_{l} \cdot x_{s,l})
$$

### Restricciones
#### Cobertura de localidades:
$$
\sum_{loc\_adj \in L} x_{s, loc\_adj} \geq 1 \quad \forall s \in S, \forall l \in L | ZC_{l, loc\_adj} = 1
$$

Esta restricción garantiza que cada localidad sea cubierta por al menos un sensor.


### Corriendo el programa sin modificaciones

Corra el modelo sin modificaciónes, evalue los resultados

In [41]:
from pyomo.environ import *
import random

# Crear el modelo
Model = ConcreteModel()

# Conjuntos
locations = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12']
sensors = ['S1', 'S2', 'S3']

# Parámetros

# Costo de consumo de energía de los sensores
energy_consumption = {'S1': 7, 'S2': 4, 'S3': 8}

# Costo de comunicación entre los sensores y las localidades
communication_cost = {
    ('L1', 'S1'): 48, ('L2', 'S1'): 38, ('L3', 'S1'): 24, ('L4', 'S1'): 17,
    ('L5', 'S1'): 30, ('L6', 'S1'): 48, ('L7', 'S1'): 28, ('L8', 'S1'): 32,
    ('L9', 'S1'): 20, ('L10', 'S1'): 20, ('L11', 'S1'): 33, ('L12', 'S1'): 45,
    ('L1', 'S2'): 49, ('L2', 'S2'): 33, ('L3', 'S2'): 12, ('L4', 'S2'): 31,
    ('L5', 'S2'): 11, ('L6', 'S2'): 33, ('L7', 'S2'): 39, ('L8', 'S2'): 47,
    ('L9', 'S2'): 11, ('L10', 'S2'): 30, ('L11', 'S2'): 42, ('L12', 'S2'): 21,
    ('L1', 'S3'): 31, ('L2', 'S3'): 34, ('L3', 'S3'): 36, ('L4', 'S3'): 37,
    ('L5', 'S3'): 25, ('L6', 'S3'): 24, ('L7', 'S3'): 12, ('L8', 'S3'): 46,
    ('L9', 'S3'): 16, ('L10', 'S3'): 30, ('L11', 'S3'): 18, ('L12', 'S3'): 48
}

# Costo de instalación en cada localidad
installation_cost = {
    'L1': 250, 'L2': 100, 'L3': 200, 'L4': 250, 'L5': 300, 'L6': 120,
    'L7': 170, 'L8': 150, 'L9': 270, 'L10': 130, 'L11': 100, 'L12': 230
}

# Cobertura de sensores en cada localidad
sensor_coverage = {
    ('L1', 'S1'): 1, ('L1', 'S2'): 0, ('L1', 'S3'): 1,
    ('L2', 'S1'): 1, ('L2', 'S2'): 0, ('L2', 'S3'): 1,
    ('L3', 'S1'): 1, ('L3', 'S2'): 0, ('L3', 'S3'): 1,
    ('L4', 'S1'): 1, ('L4', 'S2'): 1, ('L4', 'S3'): 0,
    ('L5', 'S1'): 1, ('L5', 'S2'): 1, ('L5', 'S3'): 1,
    ('L6', 'S1'): 1, ('L6', 'S2'): 1, ('L6', 'S3'): 1,
    ('L7', 'S1'): 1, ('L7', 'S2'): 0, ('L7', 'S3'): 1,
    ('L8', 'S1'): 1, ('L8', 'S2'): 1, ('L8', 'S3'): 1,
    ('L9', 'S1'): 1, ('L9', 'S2'): 1, ('L9', 'S3'): 1,
    ('L10', 'S1'): 1, ('L10', 'S2'): 1, ('L10', 'S3'): 0,
    ('L11', 'S1'): 1, ('L11', 'S2'): 1, ('L11', 'S3'): 1,
    ('L12', 'S1'): 1, ('L12', 'S2'): 0, ('L12', 'S3'): 1
}


zone_coverage = {
    ('L1', 'L1'): 1, ('L1', 'L2'): 1, ('L1', 'L3'): 1, ('L1', 'L4'): 0, ('L1', 'L5'): 1, ('L1', 'L6'): 0, ('L1', 'L7'): 0, ('L1', 'L8'): 0, ('L1', 'L9'): 0, ('L1', 'L10'): 0, ('L1', 'L11'): 0, ('L1', 'L12'): 0,
    ('L2', 'L1'): 1, ('L2', 'L2'): 1, ('L2', 'L3'): 0, ('L2', 'L4'): 0, ('L2', 'L5'): 0, ('L2', 'L6'): 0, ('L2', 'L7'): 0, ('L2', 'L8'): 0, ('L2', 'L9'): 0, ('L2', 'L10'): 0, ('L2', 'L11'): 0, ('L2', 'L12'): 0,
    ('L3', 'L1'): 1, ('L3', 'L2'): 0, ('L3', 'L3'): 1, ('L3', 'L4'): 1, ('L3', 'L5'): 1, ('L3', 'L6'): 1, ('L3', 'L7'): 1, ('L3', 'L8'): 1, ('L3', 'L9'): 0, ('L3', 'L10'): 0, ('L3', 'L11'): 0, ('L3', 'L12'): 0,
    ('L4', 'L1'): 0, ('L4', 'L2'): 0, ('L4', 'L3'): 1, ('L4', 'L4'): 1, ('L4', 'L5'): 1, ('L4', 'L6'): 1, ('L4', 'L7'): 0, ('L4', 'L8'): 1, ('L4', 'L9'): 0, ('L4', 'L10'): 0, ('L4', 'L11'): 1, ('L4', 'L12'): 0,
    ('L5', 'L1'): 1, ('L5', 'L2'): 1, ('L5', 'L3'): 1, ('L5', 'L4'): 1, ('L5', 'L5'): 1, ('L5', 'L6'): 0, ('L5', 'L7'): 0, ('L5', 'L8'): 0, ('L5', 'L9'): 0, ('L5', 'L10'): 1, ('L5', 'L11'): 1, ('L5', 'L12'): 0,
    ('L6', 'L1'): 0, ('L6', 'L2'): 0, ('L6', 'L3'): 1, ('L6', 'L4'): 1, ('L6', 'L5'): 0, ('L6', 'L6'): 1, ('L6', 'L7'): 0, ('L6', 'L8'): 1, ('L6', 'L9'): 0, ('L6', 'L10'): 0, ('L6', 'L11'): 1, ('L6', 'L12'): 0,
    ('L7', 'L1'): 0, ('L7', 'L2'): 0, ('L7', 'L3'): 1, ('L7', 'L4'): 0, ('L7', 'L5'): 0, ('L7', 'L6'): 0, ('L7', 'L7'): 1, ('L7', 'L8'): 1, ('L7', 'L9'): 0, ('L7', 'L10'): 0, ('L7', 'L11'): 0, ('L7', 'L12'): 1,
    ('L8', 'L1'): 0, ('L8', 'L2'): 0, ('L8', 'L3'): 0, ('L8', 'L4'): 1, ('L8', 'L5'): 0, ('L8', 'L6'): 1, ('L8', 'L7'): 1, ('L8', 'L8'): 1, ('L8', 'L9'): 1, ('L8', 'L10'): 0, ('L8', 'L11'): 1, ('L8', 'L12'): 1,
    ('L9', 'L1'): 0, ('L9', 'L2'): 0, ('L9', 'L3'): 0, ('L9', 'L4'): 0, ('L9', 'L5'): 0, ('L9', 'L6'): 0, ('L9', 'L7'): 0, ('L9', 'L8'): 1, ('L9', 'L9'): 1, ('L9', 'L10'): 1, ('L9', 'L11'): 1, ('L9', 'L12'): 1,
    ('L10', 'L1'): 0, ('L10', 'L2'): 0, ('L10', 'L3'): 0, ('L10', 'L4'): 0, ('L10', 'L5'): 1, ('L10', 'L6'): 0, ('L10', 'L7'): 0, ('L10', 'L8'): 0, ('L10', 'L9'): 1, ('L10', 'L10'): 1, ('L10', 'L11'): 1, ('L10', 'L12'): 0,
    ('L11', 'L1'): 0, ('L11', 'L2'): 0, ('L11', 'L3'): 0, ('L11', 'L4'): 1, ('L11', 'L5'): 1, ('L11', 'L6'): 1, ('L11', 'L7'): 0, ('L11', 'L8'): 1, ('L11', 'L9'): 1, ('L11', 'L10'): 1, ('L11', 'L11'): 1, ('L11', 'L12'): 0,
    ('L12', 'L1'): 0, ('L12', 'L2'): 0, ('L12', 'L3'): 0, ('L12', 'L4'): 0, ('L12', 'L5'): 0, ('L12', 'L6'): 0, ('L12', 'L7'): 1, ('L12', 'L8'): 1, ('L12', 'L9'): 1, ('L12', 'L10'): 0, ('L12', 'L11'): 0, ('L12', 'L12'): 1,
}



# Variables de Decisión
Model.x = Var(sensors, locations, domain=Binary)  # Si un sensor está instalado en una ubicación
Model.y = Var(sensors, domain=Binary)  # Si un sensor está instalado en cualquier ubicación

# Función Objetivo: Minimizar el costo total
def objective_rule(model):
    return (
        sum(energy_consumption[s] * model.y[s] for s in sensors) +  # Costo de energía
        sum(communication_cost[(l, s)] * model.x[s, l] for s in sensors for l in locations) +  # Costo de comunicación
        sum(installation_cost[l] * model.x[s, l] for s in sensors for l in locations)  # Costo de instalación
    )
Model.obj = Objective(rule=objective_rule, sense=minimize)


# Restricciones
def coverage_rule(model, s, l):
    if sensor_coverage[(l, s)] == 1:
        return sum(model.x[s, loc_adj] for loc_adj in locations if zone_coverage[(l, loc_adj)] == 1) >= 1
    else:
        return Constraint.Skip
Model.coverage_constraint = Constraint(sensors, locations, rule=coverage_rule)

# Resolver el modelo
solver = SolverFactory('glpk')
results = solver.solve(Model)

# Mostrar resultados
Model.display()

Model unknown

  Variables:
    x : Size=36, Index={S1, S2, S3}*{L1, L2, L3, L4, L5, L6, L7, L8, L9, L10, L11, L12}
        Key           : Lower : Value : Upper : Fixed : Stale : Domain
         ('S1', 'L1') :     0 :   0.0 :     1 : False : False : Binary
        ('S1', 'L10') :     0 :   0.0 :     1 : False : False : Binary
        ('S1', 'L11') :     0 :   1.0 :     1 : False : False : Binary
        ('S1', 'L12') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L2') :     0 :   1.0 :     1 : False : False : Binary
         ('S1', 'L3') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L4') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L5') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L6') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L7') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L8') :     0 :   1.0 :     1 : False : False : Binary
         ('S1', 'L9') :     0 : 

Evalue los resultados:

El programa minimiza correctamente el costo de cubrir todas las zonas propuestas. Esto se puede verificar haciendo el proceso a mano para cada sensor, y teniendo en cuenta el sensor_coverage.

### Corriendo el programa, teniendo en cuenta solo el costo de instalación

Corra el modelo unicamente teniendo en cuenta el costo de instalación en la función objetivo. ¿Como cambia la solución?

Para esto se modifica la función objetivo, de forma que queda de la siguiente manera:

$$
\min \space ( \sum_{l \in L} IC_{l} \cdot x_{s,l})
$$

In [42]:
from pyomo.environ import *
import random

# Crear el modelo
Model = ConcreteModel()

# Conjuntos
locations = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12']
sensors = ['S1', 'S2', 'S3']

# Parámetros

# Costo de consumo de energía de los sensores
energy_consumption = {'S1': 7, 'S2': 4, 'S3': 8}

# Costo de comunicación entre los sensores y las localidades
communication_cost = {
    ('L1', 'S1'): 48, ('L2', 'S1'): 38, ('L3', 'S1'): 24, ('L4', 'S1'): 17,
    ('L5', 'S1'): 30, ('L6', 'S1'): 48, ('L7', 'S1'): 28, ('L8', 'S1'): 32,
    ('L9', 'S1'): 20, ('L10', 'S1'): 20, ('L11', 'S1'): 33, ('L12', 'S1'): 45,
    ('L1', 'S2'): 49, ('L2', 'S2'): 33, ('L3', 'S2'): 12, ('L4', 'S2'): 31,
    ('L5', 'S2'): 11, ('L6', 'S2'): 33, ('L7', 'S2'): 39, ('L8', 'S2'): 47,
    ('L9', 'S2'): 11, ('L10', 'S2'): 30, ('L11', 'S2'): 42, ('L12', 'S2'): 21,
    ('L1', 'S3'): 31, ('L2', 'S3'): 34, ('L3', 'S3'): 36, ('L4', 'S3'): 37,
    ('L5', 'S3'): 25, ('L6', 'S3'): 24, ('L7', 'S3'): 12, ('L8', 'S3'): 46,
    ('L9', 'S3'): 16, ('L10', 'S3'): 30, ('L11', 'S3'): 18, ('L12', 'S3'): 48
}

# Costo de instalación en cada localidad
installation_cost = {
    'L1': 250, 'L2': 100, 'L3': 200, 'L4': 250, 'L5': 300, 'L6': 120,
    'L7': 170, 'L8': 150, 'L9': 270, 'L10': 130, 'L11': 100, 'L12': 230
}

# Cobertura de sensores en cada localidad
sensor_coverage = {
    ('L1', 'S1'): 1, ('L1', 'S2'): 0, ('L1', 'S3'): 1,
    ('L2', 'S1'): 1, ('L2', 'S2'): 0, ('L2', 'S3'): 1,
    ('L3', 'S1'): 1, ('L3', 'S2'): 0, ('L3', 'S3'): 1,
    ('L4', 'S1'): 1, ('L4', 'S2'): 1, ('L4', 'S3'): 0,
    ('L5', 'S1'): 1, ('L5', 'S2'): 1, ('L5', 'S3'): 1,
    ('L6', 'S1'): 1, ('L6', 'S2'): 1, ('L6', 'S3'): 1,
    ('L7', 'S1'): 1, ('L7', 'S2'): 0, ('L7', 'S3'): 1,
    ('L8', 'S1'): 1, ('L8', 'S2'): 1, ('L8', 'S3'): 1,
    ('L9', 'S1'): 1, ('L9', 'S2'): 1, ('L9', 'S3'): 1,
    ('L10', 'S1'): 1, ('L10', 'S2'): 1, ('L10', 'S3'): 0,
    ('L11', 'S1'): 1, ('L11', 'S2'): 1, ('L11', 'S3'): 1,
    ('L12', 'S1'): 1, ('L12', 'S2'): 0, ('L12', 'S3'): 1
}


zone_coverage = {
    ('L1', 'L1'): 1, ('L1', 'L2'): 1, ('L1', 'L3'): 1, ('L1', 'L4'): 0, ('L1', 'L5'): 1, ('L1', 'L6'): 0, ('L1', 'L7'): 0, ('L1', 'L8'): 0, ('L1', 'L9'): 0, ('L1', 'L10'): 0, ('L1', 'L11'): 0, ('L1', 'L12'): 0,
    ('L2', 'L1'): 1, ('L2', 'L2'): 1, ('L2', 'L3'): 0, ('L2', 'L4'): 0, ('L2', 'L5'): 0, ('L2', 'L6'): 0, ('L2', 'L7'): 0, ('L2', 'L8'): 0, ('L2', 'L9'): 0, ('L2', 'L10'): 0, ('L2', 'L11'): 0, ('L2', 'L12'): 0,
    ('L3', 'L1'): 1, ('L3', 'L2'): 0, ('L3', 'L3'): 1, ('L3', 'L4'): 1, ('L3', 'L5'): 1, ('L3', 'L6'): 1, ('L3', 'L7'): 1, ('L3', 'L8'): 1, ('L3', 'L9'): 0, ('L3', 'L10'): 0, ('L3', 'L11'): 0, ('L3', 'L12'): 0,
    ('L4', 'L1'): 0, ('L4', 'L2'): 0, ('L4', 'L3'): 1, ('L4', 'L4'): 1, ('L4', 'L5'): 1, ('L4', 'L6'): 1, ('L4', 'L7'): 0, ('L4', 'L8'): 1, ('L4', 'L9'): 0, ('L4', 'L10'): 0, ('L4', 'L11'): 1, ('L4', 'L12'): 0,
    ('L5', 'L1'): 1, ('L5', 'L2'): 1, ('L5', 'L3'): 1, ('L5', 'L4'): 1, ('L5', 'L5'): 1, ('L5', 'L6'): 0, ('L5', 'L7'): 0, ('L5', 'L8'): 0, ('L5', 'L9'): 0, ('L5', 'L10'): 1, ('L5', 'L11'): 1, ('L5', 'L12'): 0,
    ('L6', 'L1'): 0, ('L6', 'L2'): 0, ('L6', 'L3'): 1, ('L6', 'L4'): 1, ('L6', 'L5'): 0, ('L6', 'L6'): 1, ('L6', 'L7'): 0, ('L6', 'L8'): 1, ('L6', 'L9'): 0, ('L6', 'L10'): 0, ('L6', 'L11'): 1, ('L6', 'L12'): 0,
    ('L7', 'L1'): 0, ('L7', 'L2'): 0, ('L7', 'L3'): 1, ('L7', 'L4'): 0, ('L7', 'L5'): 0, ('L7', 'L6'): 0, ('L7', 'L7'): 1, ('L7', 'L8'): 1, ('L7', 'L9'): 0, ('L7', 'L10'): 0, ('L7', 'L11'): 0, ('L7', 'L12'): 1,
    ('L8', 'L1'): 0, ('L8', 'L2'): 0, ('L8', 'L3'): 0, ('L8', 'L4'): 1, ('L8', 'L5'): 0, ('L8', 'L6'): 1, ('L8', 'L7'): 1, ('L8', 'L8'): 1, ('L8', 'L9'): 1, ('L8', 'L10'): 0, ('L8', 'L11'): 1, ('L8', 'L12'): 1,
    ('L9', 'L1'): 0, ('L9', 'L2'): 0, ('L9', 'L3'): 0, ('L9', 'L4'): 0, ('L9', 'L5'): 0, ('L9', 'L6'): 0, ('L9', 'L7'): 0, ('L9', 'L8'): 1, ('L9', 'L9'): 1, ('L9', 'L10'): 1, ('L9', 'L11'): 1, ('L9', 'L12'): 1,
    ('L10', 'L1'): 0, ('L10', 'L2'): 0, ('L10', 'L3'): 0, ('L10', 'L4'): 0, ('L10', 'L5'): 1, ('L10', 'L6'): 0, ('L10', 'L7'): 0, ('L10', 'L8'): 0, ('L10', 'L9'): 1, ('L10', 'L10'): 1, ('L10', 'L11'): 1, ('L10', 'L12'): 0,
    ('L11', 'L1'): 0, ('L11', 'L2'): 0, ('L11', 'L3'): 0, ('L11', 'L4'): 1, ('L11', 'L5'): 1, ('L11', 'L6'): 1, ('L11', 'L7'): 0, ('L11', 'L8'): 1, ('L11', 'L9'): 1, ('L11', 'L10'): 1, ('L11', 'L11'): 1, ('L11', 'L12'): 0,
    ('L12', 'L1'): 0, ('L12', 'L2'): 0, ('L12', 'L3'): 0, ('L12', 'L4'): 0, ('L12', 'L5'): 0, ('L12', 'L6'): 0, ('L12', 'L7'): 1, ('L12', 'L8'): 1, ('L12', 'L9'): 1, ('L12', 'L10'): 0, ('L12', 'L11'): 0, ('L12', 'L12'): 1,
}



# Variables de Decisión
Model.x = Var(sensors, locations, domain=Binary)  # Si un sensor está instalado en una ubicación
Model.y = Var(sensors, domain=Binary)  # Si un sensor está instalado en cualquier ubicación

# Función Objetivo: Minimizar el costo total
def objective_rule(model):
    return (
        sum(installation_cost[l] * model.x[s, l] for s in sensors for l in locations)  # Aquí solo se tiene en cuenta el costo de instalación
    )
Model.obj = Objective(rule=objective_rule, sense=minimize)


# Restricciones
def coverage_rule(model, s, l):
    if sensor_coverage[(l, s)] == 1:
        return sum(model.x[s, loc_adj] for loc_adj in locations if zone_coverage[(l, loc_adj)] == 1) >= 1
    else:
        return Constraint.Skip
Model.coverage_constraint = Constraint(sensors, locations, rule=coverage_rule)

# Resolver el modelo
solver = SolverFactory('glpk')
results = solver.solve(Model)

# Mostrar resultados
Model.display()

Model unknown

  Variables:
    x : Size=36, Index={S1, S2, S3}*{L1, L2, L3, L4, L5, L6, L7, L8, L9, L10, L11, L12}
        Key           : Lower : Value : Upper : Fixed : Stale : Domain
         ('S1', 'L1') :     0 :   0.0 :     1 : False : False : Binary
        ('S1', 'L10') :     0 :   0.0 :     1 : False : False : Binary
        ('S1', 'L11') :     0 :   1.0 :     1 : False : False : Binary
        ('S1', 'L12') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L2') :     0 :   1.0 :     1 : False : False : Binary
         ('S1', 'L3') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L4') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L5') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L6') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L7') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L8') :     0 :   1.0 :     1 : False : False : Binary
         ('S1', 'L9') :     0 : 

¿Como cambia la solución?

Al correr el programa teniendo en cuenta unicamente el costo de instalación en la función objetivo, se obtiene el mismo resultado. Esto tiene sentido, debido a que los costos de instalación son los mas altos respecto a los demás costos. En otras palabras, es el costo dominante que determina el comportamiento de la función objetivo.